<img src="../images/GeoCAT_logo.png" width=400 alt="GeoCAT Logo"></img>

# Advanced Plotting with GeoCAT-Examples

---

## Learning Objectives
- Navigating GeoCAT-examples gallery
- Recreating an advanced matplotlib example with GeoCAT-examples
- Creating an animated plot

## Prerequisites


| Concepts | Importance |
| --- | --- |
| Basic familiarity with NumPy | Necessary |
| Basic familiarity with Matplotlib | Necessary |
| Basic familiarity with Cartopy | Helpful |
| Basic familiarity with Xarray | Helpful |

- **Time to learn**: *15-20 minutes*


---

## Imports

In [ ]:
import cartopy.crs as ccrs
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt

In [1]:
# geocat specific imports
import geocat.datafiles as gdf

## Get the data we want to animate
If you're replicating a script from geocat-examples, there's a good chance that we've stored the required data files in a repository called [geocat-datafiles](https://github.com/NCAR/geocat-datafiles), which we can pull from.

In [ ]:
# Open a netCDF data file using xarray default engine and load the data into xarrays
ds = xr.open_dataset(gdf.get("netcdf_files/meccatemp.cdf"), decode_times=False)

# Pull out surface temperature Dataarray from xarray Dataset
tas = ds.t

In [ ]:
tas

So, we have a latitude longitude grid over 31 time slices.

## Initial vizualization
To get an idea about what we're working with, let's plot the first time slice and figure out how we want to fomat the plot before we get into the animation

Let's use xarray to plot the first time slice as a contour plot.

In [ ]:
fig = plt.figure()
tas[0,:,:].plot.contourf(levels=60)

Now, let's redo that, add some formatting to the plot. Here's a list of suggested changes that I'm going to implement in the next step:
- adding a projection and map overlay
- changing the axes and colorbar labels
- changing the colormap
- changing the size and positioning of the colorbar

In [ ]:
fig = plt.figure()

# explicitly set up axis with projection
ax = plt.axes(projection=ccrs.PlateCarree())

# add coastlines
ax.coastlines(linewidths=0.5)

tas[0, :, :].plot.contourf(ax=ax,
                           transform=ccrs.PlateCarree(),
                           levels=60,
                           cmap="inferno",
                           cbar_kwargs={
                               "orientation": "horizontal",
                               "label": "t [K]"}
                            )

# configure axes
ax.set_xlabel("")
ax.set_xticks(np.arange(-180, 181, 30))

ax.set_ylabel("")
ax.set_yticks(np.arange(-90, 91, 30));

Now that we have somewhere to start, let's get in to the animation.

## Animation using matplotlib

We are going to be using matplotlib's [`FuncAnimation`](https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.FuncAnimation.html), which makes animations by repeatedly calling a function. Using this method involves three main steps:

1. Create an initial state of the plot
1. Make a function that can "progress" the plot to the next frame of the animation
1. Create the animation using `FuncAnimation`

See [matplotlib's animation documentation](https://matplotlib.org/stable/api/animation_api.html) for more information.

In [ ]:
from matplotlib.animation import FuncAnimation

### Step 1: Initial State

We're going to use the figure above as the step one "intial state" for the animation. An initilization function can be created separately and passed in as an argument to `FuncAnimation`, but it is optional.

### Step 2: Animation progression function
For each frame in the animation, we want to progress time index. The simplest way to achieve this is to make the progression function plot each time slice with the desired formatting.

Note that some of the formatting will have to be set after the plotting call, as the plotting call will override the user-specified formatting.

Also, if you are using a colorbar, it is important to establish the colorbar in the initilazation and set kwarg `add_colorbar` to be false. Otherwise, an additional colorbar will be added on each iteration.

In [ ]:
def update(i):
    tas[i, :, :].plot.contourf(ax=ax,
                       transform=ccrs.PlateCarree(),
                       levels=60,
                       cmap="inferno",
                       add_colorbar=False
                        )
    # configure axes
    
    ax.set_xlabel("")
    ax.set_xticks(np.arange(-180, 181, 30))

    ax.set_ylabel("")
    ax.set_yticks(np.arange(-90, 91, 30))
    
    # Set plot title
    ax.set_title("January Surface Temperatures (K) - Day " + str(i+1));

### Step 3: `FuncAnimation`
Now, we are going to create the animation. Note that when using `FuncAnimation`, it is important to save the output, even if you will never use it, as it is at risk of being collected by Python's garbage collector if not saved to an instance.

In [ ]:
# runs the animation
anim = FuncAnimation(fig, update, frames=31)

To show the animation in this jupyter notebook, we need to set the rc parameter for animation to `html5`, instead of the default, which is `none`.

In [ ]:
from matplotlib import rc
rc('animation', html='html5')
anim

Now, we can save the animation using `FuncAnimation.save()`. This step can take several minutes. We can also set a higher resolution with `dpi`.

In [ ]:
anim.save('animate.gif', writer='pillow', fps=5, dpi=300);

---